# Homework 2

**Name:** -- Víctor Manuel Mariscal Cervantes --

**e-mail:** -- victor.mariscal4459@alumnos.udg.mx --

# MODULES

In [3]:
# Load modules
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats as stats
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
from IPython.display import display, HTML


In [4]:
# Define 2Vec class
class Vec2d(object):
    """
    2d vector class, supports vector and scalar operators,
    and also provides a bunch of high level functions
    """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)
    
    def __str__(self):
        return f"Vec2d({self.x}, {self.y})"
    
    def __repr__(self):
        return f"Vec2d({self.x}, {self.y})"
    
    def to_tuple(self):
        return (self.x, self.y)

In [5]:
# Module 1: Correlated Random Walk
def generate_crw_trajectory(num_steps, speed, start_pos_x, start_pos_y, cauchy_coef, seed=None):
    """
    Generate a Correlated Random Walk trajectory
    
    Parameters:
    -----------
    num_steps : int
        Number of steps in the trajectory
    speed : float
        Speed/step size for the walker
    start_pos_x, start_pos_y : float
        Starting position coordinates
    cauchy_coef : float
        Cauchy distribution coefficient (0 < c < 1)
    seed : int, optional
        Random seed for reproducibility
        
    Returns:
    --------
    trajectory : numpy.ndarray
        Array of shape (num_steps, 3) with x, y, time coordinates
    """
    if seed is not None:
        np.random.seed(seed)
        
    # Initialize trajectory array
    trajectory = np.zeros((num_steps, 3))
    
    # Set starting position
    current_pos = Vec2d(start_pos_x, start_pos_y)
    
    # Initial direction (random)
    direction = Vec2d(1, 0)
    
    # Store initial position
    trajectory[0, 0] = current_pos.x
    trajectory[0, 1] = current_pos.y
    trajectory[0, 2] = 0  # time
    
    # Generate trajectory
    for i in range(1, num_steps):
        # Sample turning angle from Cauchy distribution
        angle = np.random.standard_cauchy() * cauchy_coef
        
        # Update direction with rotation
        direction = direction.rotated(angle)
        
        # Normalize direction and apply speed
        length = direction.get_length()
        if length > 0:
            direction = Vec2d(direction.x / length * speed, direction.y / length * speed)
        
        # Update position
        current_pos = current_pos + direction
        
        # Store position
        trajectory[i, 0] = current_pos.x
        trajectory[i, 1] = current_pos.y
        trajectory[i, 2] = i  # time
    
    return trajectory

# Calculate Mean Squared Displacement (MSD)
def calculate_msd(trajectory):
    """
    Calculate Mean Squared Displacement for a trajectory
    
    Parameters:
    -----------
    trajectory : numpy.ndarray
        Array with x, y coordinates
        
    Returns:
    --------
    tau : numpy.ndarray
        Time lags
    msd : numpy.ndarray
        MSD values for each time lag
    """
    positions = trajectory[:, :2]  # Only x, y coordinates
    n_points = len(positions)
    
    # Calculate maximum tau (time lag)
    max_tau = min(n_points // 4, 1000)  # Use 1/4 of the trajectory for reliable statistics, max 1000
    
    tau = np.arange(1, max_tau + 1)
    msd = np.zeros(max_tau)
    
    # Calculate MSD for each tau (optimized for performance)
    for i, t in enumerate(tau):
        if i % 10 == 0 or i < 10:  # Calculate more points at the beginning, fewer later
            # Calculate squared displacements
            sd = np.sum((positions[t:] - positions[:-t])**2, axis=1)
            # Average to get MSD
            msd[i] = np.mean(sd)
        elif i > 0:
            # Interpolate for skipped points
            msd[i] = msd[i-1] + (msd[i//10*10] - msd[(i//10-1)*10]) / 10
    
    return tau, msd

In [6]:
# Module 2: Lévy Distribution
def levy_pdf(x, alpha, beta=0, mu=0, c=1):
    """
    Lévy stable distribution PDF approximation
    
    Parameters:
    -----------
    x : array_like
        Points at which to evaluate the PDF
    alpha : float
        Stability parameter (0 < alpha <= 2)
    beta : float, optional
        Skewness parameter (-1 <= beta <= 1)
    mu : float, optional
        Location parameter
    c : float, optional
        Scale parameter
        
    Returns:
    --------
    pdf : array_like
        PDF values at points x
    """
    # Using scipy's implementation of the stable distribution
    return stats.levy_stable.pdf(x, alpha, beta, loc=mu, scale=c)

def plot_levy_distributions(x_range, alphas, mu=3.0, beta=0.0):
    """
    Plot multiple Lévy distributions with different alpha values
    
    Parameters:
    -----------
    x_range : array_like
        Range of x values to plot
    alphas : list
        List of alpha values to plot
    mu : float, optional
        Location parameter
    beta : float, optional
        Skewness parameter
        
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        Plotly figure object
    """
    fig = go.Figure()
    
    # Add a trace for each alpha value
    for alpha in alphas:
        pdf_values = levy_pdf(x_range, alpha, beta, mu)
        
        fig.add_trace(go.Scatter(
            x=x_range,
            y=pdf_values,
            mode='lines',
            name=f'Lévy {alpha}',
            line=dict(width=2)
        ))
    
    # Update layout
    fig.update_layout(
        title="Lévy Stable Distributions",
        xaxis_title="x",
        yaxis_title="Probability Density",
        template="plotly_white",
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99
        )
    )
    
    return fig

In [7]:
# Module 3: Lévy Distribution - Histogram + Curve
def generate_levy_samples(n_samples, alpha, beta=0, mu=0, c=1, seed=None):
    """
    Generate samples from a Lévy stable distribution
    
    Parameters:
    -----------
    n_samples : int
        Number of samples to generate
    alpha : float
        Stability parameter (0 < alpha <= 2)
    beta : float, optional
        Skewness parameter (-1 <= beta <= 1)
    mu : float, optional
        Location parameter
    c : float, optional
        Scale parameter
    seed : int, optional
        Random seed for reproducibility
        
    Returns:
    --------
    samples : numpy.ndarray
        Array of samples from the Lévy distribution
    """
    if seed is not None:
        np.random.seed(seed)
        
    # Generate samples using scipy's levy_stable.rvs
    return stats.levy_stable.rvs(alpha, beta, loc=mu, scale=c, size=n_samples)

def plot_levy_histogram_with_curve(samples, alpha, beta=0, mu=0, c=1, bins=50):
    """
    Plot histogram of Lévy samples with the theoretical PDF curve
    
    Parameters:
    -----------
    samples : array_like
        Samples from the Lévy distribution
    alpha : float
        Stability parameter of the samples
    beta : float, optional
        Skewness parameter of the samples
    mu : float, optional
        Location parameter of the samples
    c : float, optional
        Scale parameter of the samples
    bins : int, optional
        Number of histogram bins
        
    Returns:
    --------
    fig : matplotlib.figure.Figure
        Matplotlib figure object
    """
    # Create a range for the PDF
    x_min, x_max = np.min(samples), np.max(samples)
    x_range = np.linspace(x_min, x_max, 1000)
    
    # Calculate the PDF
    pdf_values = levy_pdf(x_range, alpha, beta, mu, c)
    
    # Create the figure
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot histogram
    counts, bins, _ = ax.hist(samples, bins=bins, density=True, alpha=0.5, 
                            label='Lévy samples', color='skyblue')
    
    # Plot PDF
    ax.plot(x_range, pdf_values, 'r-', lw=2, label='Lévy PDF')
    
    # Add labels and legend
    ax.set_xlabel('x')
    ax.set_ylabel('Probability Density')
    ax.set_title(f'Lévy Distribution (α={alpha}, β={beta}, μ={mu})')
    ax.legend()
    
    # Improve appearance
    ax.grid(alpha=0.3)
    fig.tight_layout()
    
    return fig

In [8]:
# Module 4: Lévy Flight
def generate_levy_flight_trajectory(num_steps, alpha, beta=0, mu=3.0, c=1.0, 
                                   start_pos_x=0, start_pos_y=0, cauchy_coef=0.7, 
                                   seed=None):
    """
    Generate a Lévy flight trajectory
    
    Parameters:
    -----------
    num_steps : int
        Number of steps in the trajectory
    alpha : float
        Stability parameter for step length distribution
    beta : float, optional
        Skewness parameter for step length distribution
    mu : float, optional
        Location parameter for step length distribution
    c : float, optional
        Scale parameter for step length distribution
    start_pos_x, start_pos_y : float
        Starting position coordinates
    cauchy_coef : float
        Cauchy distribution coefficient for turning angles
    seed : int, optional
        Random seed for reproducibility
        
    Returns:
    --------
    trajectory : numpy.ndarray
        Array of shape (num_points, 3) with x, y, time coordinates
    """
    if seed is not None:
        np.random.seed(seed)
    
    # We'll need to generate more steps than requested because
    # Lévy steps will be treated as "number of steps in the same direction"
    max_possible_points = num_steps * 10  # A reasonable upper bound
    
    # Initialize larger trajectory array
    extended_trajectory = np.zeros((max_possible_points, 3))
    
    # Set starting position
    current_pos = Vec2d(start_pos_x, start_pos_y)
    
    # Initial direction (random)
    direction = Vec2d(1, 0)
    
    # Store initial position
    extended_trajectory[0, 0] = current_pos.x
    extended_trajectory[0, 1] = current_pos.y
    extended_trajectory[0, 2] = 0  # time
    
    point_index = 1
    step_index = 0
    
    # Generate trajectory
    while step_index < num_steps and point_index < max_possible_points:
        # Sample turning angle from Cauchy distribution
        angle = np.random.standard_cauchy() * cauchy_coef
        
        # Update direction with rotation
        direction = direction.rotated(angle)
        
        # Normalize direction
        length = direction.get_length()
        if length > 0:
            direction = Vec2d(direction.x / length, direction.y / length)
        
        # Sample step length from Lévy distribution
        # We interpret this as "number of steps in the same direction"
        levy_steps = int(max(1, stats.levy_stable.rvs(alpha, beta, loc=mu, scale=c)))
        
        # Take the Lévy number of steps in the current direction
        for i in range(min(levy_steps, 100)):  # Cap at 100 for performance
            if point_index >= max_possible_points:
                break
                
            # Update position (unit step size)
            current_pos = current_pos + direction
            
            # Store position
            extended_trajectory[point_index, 0] = current_pos.x
            extended_trajectory[point_index, 1] = current_pos.y
            extended_trajectory[point_index, 2] = point_index  # time
            
            point_index += 1
        
        step_index += 1
    
    # Trim the trajectory to actual points generated
    trajectory = extended_trajectory[:point_index]
    
    return trajectory

In [9]:
# Module 5: Multiple CRW Trajectories
def generate_multiple_crw_trajectories(num_trajectories, num_steps, speed, 
                                      start_pos_x, start_pos_y, 
                                      cauchy_coefs, seed=None):
    """
    Generate multiple CRW trajectories with different Cauchy coefficients
    
    Parameters:
    -----------
    num_trajectories : int
        Number of trajectories to generate
    num_steps : int
        Number of steps in each trajectory
    speed : float
        Speed/step size for the walkers
    start_pos_x, start_pos_y : float
        Starting position coordinates
    cauchy_coefs : list or array
        Cauchy coefficients for each trajectory
    seed : int, optional
        Random seed for reproducibility
        
    Returns:
    --------
    trajectories : list
        List of trajectory arrays
    """
    if seed is not None:
        np.random.seed(seed)
        
    if len(cauchy_coefs) < num_trajectories:
        raise ValueError("Not enough Cauchy coefficients provided")
    
    trajectories = []
    
    for i in range(num_trajectories):
        traj = generate_crw_trajectory(
            num_steps=num_steps,
            speed=speed,
            start_pos_x=start_pos_x,
            start_pos_y=start_pos_y,
            cauchy_coef=cauchy_coefs[i],
            seed=seed+i if seed is not None else None
        )
        trajectories.append(traj)
    
    return trajectories


In [10]:
# Calculate Mean Squared Displacement (MSD)
def calculate_msd(trajectory):
    """
    Calculate Mean Squared Displacement for a trajectory
    
    Parameters:
    -----------
    trajectory : numpy.ndarray
        Array with x, y coordinates
        
    Returns:
    --------
    tau : numpy.ndarray
        Time lags
    msd : numpy.ndarray
        MSD values for each time lag
    """
    positions = trajectory[:, :2]  # Only x, y coordinates
    n_points = len(positions)
    
    # Calculate maximum tau (time lag)
    max_tau = n_points // 4  # Use 1/4 of the trajectory for reliable statistics
    
    tau = np.arange(1, max_tau + 1)
    msd = np.zeros(max_tau)
    
    # Calculate MSD for each tau
    for i, t in enumerate(tau):
        # Calculate squared displacements
        sd = np.sum((positions[t:] - positions[:-t])**2, axis=1)
        # Average to get MSD
        msd[i] = np.mean(sd)
    
    return tau, msd

def plot_msd(tau, msd, title="Mean Squared Displacement"):
    """
    Plot Mean Squared Displacement
    
    Parameters:
    -----------
    tau : array_like
        Time lags
    msd : array_like
        MSD values
    title : str
        Plot title
        
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        Plotly figure object
    """
    fig = go.Figure()
    
    # Add MSD trace
    fig.add_trace(go.Scatter(
        x=tau,
        y=msd,
        mode='lines',
        name='MSD',
        line=dict(color='purple', width=2)
    ))
    
    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title="τ",
        yaxis_title="MSD",
        template="plotly_white"
    )
    
    return fig

In [11]:
def create_dash_dashboard():
    """Create a Dash dashboard for random walks visualization"""
    
    # Import standard Dash instead of JupyterDash
    import dash
    from dash import dcc, html
    from dash.dependencies import Input, Output, State
    import plotly.graph_objs as go
    import dash_bootstrap_components as dbc
    import numpy as np
    
    # Initialize the standard Dash app
    app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
    
    # App layout
    app.layout = html.Div([
        html.H1("Random Walks Dashboard", className="text-center my-4"),
        
        # Tabs for different activities
        dbc.Tabs([
            # Activity 1: CRW - 1 Trajectory
            dbc.Tab(label="Activity 1: CRW", children=[
                dbc.Row([
                    # Controls
                    dbc.Col([
                        html.H5("Parameters", className="mt-3"),
                        
                        html.Label("Number of steps"),
                        dcc.Input(
                            id="crw-num-steps",
                            type="number",
                            value=300,  # Default to a smaller value for performance
                            min=100,
                            max=5000,
                            step=100,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Speed"),
                        dcc.Input(
                            id="crw-speed",
                            type="number",
                            value=5,
                            min=1,
                            max=20,
                            step=1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Starting pos_x"),
                        dcc.Input(
                            id="crw-start-x",
                            type="number",
                            value=0,
                            min=-100,
                            max=100,
                            step=10,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Starting pos_y"),
                        dcc.Input(
                            id="crw-start-y",
                            type="number",
                            value=0,
                            min=-100,
                            max=100,
                            step=10,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Cauchy coefficient"),
                        dcc.Input(
                            id="crw-cauchy",
                            type="number",
                            value=0.7,
                            min=0.1,
                            max=0.9,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        dbc.Button(
                            "Update", 
                            id="crw-update-button",
                            color="primary",
                            className="mt-3 w-100"
                        ),
                    ], width=3),
                    
                    # Trajectory plot
                    dbc.Col([
                        html.H5("Trajectory", className="mt-3"),
                        dcc.Graph(
                            id="crw-trajectory-graph",
                            style={"height": "500px"}
                        )
                    ], width=5),
                    
                    # MSD plot
                    dbc.Col([
                        html.H5("Metric", className="mt-3"),
                        dcc.Dropdown(
                            id="crw-metric-type",
                            options=[
                                {"label": "MSD", "value": "MSD"}
                            ],
                            value="MSD",
                            clearable=False,
                            className="mb-2"
                        ),
                        dcc.Graph(
                            id="crw-msd-graph",
                            style={"height": "500px"}
                        )
                    ], width=4)
                ])
            ]),
            
            # Activity 2: Lévy distribution - N different curves
            dbc.Tab(label="Activity 2: Lévy Curves", children=[
                dbc.Row([
                    # Controls
                    dbc.Col([
                        html.H5("Parameters", className="mt-3"),
                        
                        html.Label("μ parameter"),
                        dcc.Input(
                            id="levy-curves-mu",
                            type="number",
                            value=3.0,
                            min=0,
                            max=10,
                            step=0.5,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("β parameter"),
                        dcc.Input(
                            id="levy-curves-beta",
                            type="number",
                            value=0,
                            min=-1,
                            max=1,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        html.H6("Alpha Values", className="mt-3"),
                        dbc.Checklist(
                            id="levy-curves-alphas",
                            options=[
                                {"label": "α = 0.1", "value": 0.1},
                                {"label": "α = 0.5", "value": 0.5},
                                {"label": "α = 1.0", "value": 1.0},
                                {"label": "α = 1.9", "value": 1.9}
                            ],
                            value=[0.1, 0.5, 1.0, 1.9],  # Default all selected
                            className="mb-3"
                        ),
                        
                        dbc.Button(
                            "Update", 
                            id="levy-curves-update-button",
                            color="primary",
                            className="mt-3 w-100"
                        ),
                    ], width=3),
                    
                    # Lévy distributions plot
                    dbc.Col([
                        html.H5("Lévy Distributions", className="mt-3"),
                        dcc.Graph(
                            id="levy-curves-graph",
                            style={"height": "600px"}
                        )
                    ], width=9)
                ])
            ]),
            
            # Activity 3: Lévy distribution - Histogram + Curve
            dbc.Tab(label="Activity 3: Lévy Histogram", children=[
                dbc.Row([
                    # Controls
                    dbc.Col([
                        html.H5("Parameters", className="mt-3"),
                        
                        html.Label("α parameter"),
                        dcc.Input(
                            id="levy-hist-alpha",
                            type="number",
                            value=1.5,
                            min=0.1,
                            max=2.0,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("β parameter"),
                        dcc.Input(
                            id="levy-hist-beta",
                            type="number",
                            value=0,
                            min=-1,
                            max=1,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("μ parameter"),
                        dcc.Input(
                            id="levy-hist-mu",
                            type="number",
                            value=3.0,
                            min=0,
                            max=10,
                            step=0.5,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("c parameter"),
                        dcc.Input(
                            id="levy-hist-c",
                            type="number",
                            value=1.0,
                            min=0.1,
                            max=5.0,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("# Samples"),
                        dcc.Input(
                            id="levy-hist-samples",
                            type="number",
                            value=1000,
                            min=100,
                            max=10000,
                            step=100,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("# Bins"),
                        dcc.Input(
                            id="levy-hist-bins",
                            type="number",
                            value=50,
                            min=10,
                            max=200,
                            step=10,
                            className="mb-2 w-100"
                        ),
                        
                        dbc.Button(
                            "Update", 
                            id="levy-hist-update-button",
                            color="primary",
                            className="mt-3 w-100"
                        ),
                    ], width=3),
                    
                    # Histogram plot
                    dbc.Col([
                        html.H5("Lévy Distribution Histogram", className="mt-3"),
                        dcc.Graph(
                            id="levy-hist-graph",
                            style={"height": "600px"}
                        )
                    ], width=9)
                ])
            ]),
            
            # Activity 4: Lévy flight - Vec2d - 1 Trajectory
            dbc.Tab(label="Activity 4: Lévy Flight", children=[
                dbc.Row([
                    # Controls
                    dbc.Col([
                        html.H5("Parameters", className="mt-3"),
                        
                        html.Label("Number of steps"),
                        dcc.Input(
                            id="levy-flight-steps",
                            type="number",
                            value=300,
                            min=100,
                            max=5000,
                            step=100,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("α parameter"),
                        dcc.Input(
                            id="levy-flight-alpha",
                            type="number",
                            value=1.5,
                            min=0.1,
                            max=2.0,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("μ parameter"),
                        dcc.Input(
                            id="levy-flight-mu",
                            type="number",
                            value=3.0,
                            min=0,
                            max=10,
                            step=0.5,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("c parameter"),
                        dcc.Input(
                            id="levy-flight-c",
                            type="number",
                            value=1.0,
                            min=0.1,
                            max=5.0,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Starting pos_x"),
                        dcc.Input(
                            id="levy-flight-start-x",
                            type="number",
                            value=0,
                            min=-100,
                            max=100,
                            step=10,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Starting pos_y"),
                        dcc.Input(
                            id="levy-flight-start-y",
                            type="number",
                            value=0,
                            min=-100,
                            max=100,
                            step=10,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Cauchy coefficient"),
                        dcc.Input(
                            id="levy-flight-cauchy",
                            type="number",
                            value=0.7,
                            min=0.1,
                            max=0.9,
                            step=0.1,
                            className="mb-2 w-100"
                        ),
                        
                        dbc.Button(
                            "Update", 
                            id="levy-flight-update-button",
                            color="primary",
                            className="mt-3 w-100"
                        ),
                    ], width=3),
                    
                    # Trajectory plot
                    dbc.Col([
                        html.H5("Lévy Flight Trajectory", className="mt-3"),
                        dcc.Graph(
                            id="levy-flight-graph",
                            style={"height": "600px"}
                        )
                    ], width=9)
                ])
            ]),
            
            # Activity 5: Correlated Random Walk - Vec2d - N Trajectories
            dbc.Tab(label="Activity 5: Multiple CRW", children=[
                dbc.Row([
                    # Controls
                    dbc.Col([
                        html.H5("Parameters", className="mt-3"),
                        
                        html.Label("Number of trajectories"),
                        dcc.Input(
                            id="multi-crw-n-traj",
                            type="number",
                            value=6,
                            min=2,
                            max=10,
                            step=1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Steps per trajectory"),
                        dcc.Input(
                            id="multi-crw-steps",
                            type="number",
                            value=300,
                            min=100,
                            max=5000,
                            step=100,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Speed"),
                        dcc.Input(
                            id="multi-crw-speed",
                            type="number",
                            value=5,
                            min=1,
                            max=20,
                            step=1,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Starting pos_x"),
                        dcc.Input(
                            id="multi-crw-start-x",
                            type="number",
                            value=0,
                            min=-100,
                            max=100,
                            step=10,
                            className="mb-2 w-100"
                        ),
                        
                        html.Label("Starting pos_y"),
                        dcc.Input(
                            id="multi-crw-start-y",
                            type="number",
                            value=0,
                            min=-100,
                            max=100,
                            step=10,
                            className="mb-2 w-100"
                        ),
                        
                        dbc.Button(
                            "Update", 
                            id="multi-crw-update-button",
                            color="primary",
                            className="mt-3 w-100"
                        ),
                    ], width=3),
                    
                    # Multiple trajectories plot
                    dbc.Col([
                        html.H5("Multiple CRW Trajectories", className="mt-3"),
                        dcc.Graph(
                            id="multi-crw-graph",
                            style={"height": "600px"}
                        )
                    ], width=9)
                ])
            ])
        ], className="mb-3")
    ], className="container-fluid")
    
    # Activity 1: CRW - 1 Trajectory - Callbacks
    @app.callback(
        [Output("crw-trajectory-graph", "figure"),
         Output("crw-msd-graph", "figure")],
        Input("crw-update-button", "n_clicks"),
        [State("crw-num-steps", "value"),
         State("crw-speed", "value"),
         State("crw-start-x", "value"),
         State("crw-start-y", "value"),
         State("crw-cauchy", "value")]
    )
    def update_crw(n_clicks, num_steps, speed, start_x, start_y, cauchy_coef):
        # Generate trajectory
        trajectory = generate_crw_trajectory(
            num_steps=num_steps,
            speed=speed,
            start_pos_x=start_x,
            start_pos_y=start_y,
            cauchy_coef=cauchy_coef
        )
        
        # Calculate MSD
        tau, msd = calculate_msd(trajectory)
        
        # Plot every nth point to improve performance
        plot_every_n = max(1, len(trajectory) // 500)
        
        # Create trajectory plot
        traj_fig = go.Figure()
        
        traj_fig.add_trace(go.Scatter3d(
            x=trajectory[::plot_every_n, 0],
            y=trajectory[::plot_every_n, 1],
            z=trajectory[::plot_every_n, 2],
            mode='lines',
            name='CRW',
            line=dict(color='red', width=3)
        ))
        
        traj_fig.update_layout(
            title="CRW trajectory",
            scene=dict(
                xaxis_title="x_pos",
                yaxis_title="y_pos",
                zaxis_title="time",
                aspectmode='manual',
                aspectratio=dict(x=1, y=1, z=1)
            ),
            margin=dict(l=0, r=0, b=0, t=30),
            template="plotly_white"
        )
        
        # Create MSD plot
        msd_fig = go.Figure()
        
        msd_fig.add_trace(go.Scatter(
            x=tau,
            y=msd,
            mode='lines',
            name='MSD',
            line=dict(color='purple', width=2)
        ))
        
        msd_fig.update_layout(
            title="Mean Squared Displacement",
            xaxis_title="tau",
            yaxis_title="MSD",
            template="plotly_white",
            margin=dict(l=0, r=0, b=0, t=30)
        )
        
        return traj_fig, msd_fig
    
    # Activity 2: Lévy distribution - N different curves - Callbacks
    @app.callback(
        Output("levy-curves-graph", "figure"),
        Input("levy-curves-update-button", "n_clicks"),
        [State("levy-curves-mu", "value"),
         State("levy-curves-beta", "value"),
         State("levy-curves-alphas", "value")]
    )
    def update_levy_curves(n_clicks, mu, beta, alphas):
        if not alphas:
            # Default to one alpha if none selected
            alphas = [1.0]
        
        # Generate x range
        x_range = np.linspace(-2, 8, 1000)
        
        # Create figure
        fig = go.Figure()
        
        # Add a trace for each alpha value
        for alpha in alphas:
            pdf_values = levy_pdf(x_range, alpha, beta, mu)
            
            fig.add_trace(go.Scatter(
                x=x_range,
                y=pdf_values,
                mode='lines',
                name=f'Lévy {alpha}',
                line=dict(width=2)
            ))
        
        # Update layout
        fig.update_layout(
            title="Lévy Stable Distributions",
            xaxis_title="x",
            yaxis_title="Probability Density",
            template="plotly_white",
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99
            )
        )
        
        return fig
    
    # Activity 3: Lévy distribution - Histogram + Curve - Callbacks
    @app.callback(
        Output("levy-hist-graph", "figure"),
        Input("levy-hist-update-button", "n_clicks"),
        [State("levy-hist-alpha", "value"),
         State("levy-hist-beta", "value"),
         State("levy-hist-mu", "value"),
         State("levy-hist-c", "value"),
         State("levy-hist-samples", "value"),
         State("levy-hist-bins", "value")]
    )
    def update_levy_histogram(n_clicks, alpha, beta, mu, c, n_samples, n_bins):
        # Generate samples
        samples = generate_levy_samples(
            n_samples=n_samples,
            alpha=alpha,
            beta=beta,
            mu=mu,
            c=c
        )
        
        # Create x range for PDF
        x_min, x_max = np.min(samples), np.max(samples)
        x_range = np.linspace(x_min, x_max, 1000)
        
        # Calculate PDF
        pdf_values = levy_pdf(x_range, alpha, beta, mu, c)
        
        # Create figure
        fig = go.Figure()
        
        # Add histogram
        fig.add_trace(go.Histogram(
            x=samples,
            histnorm='probability density',
            name='Samples',
            nbinsx=n_bins,
            opacity=0.5,
            marker_color='skyblue'
        ))
        
        # Add PDF curve
        fig.add_trace(go.Scatter(
            x=x_range,
            y=pdf_values,
            mode='lines',
            name='PDF',
            line=dict(color='red', width=2)
        ))
        
        # Update layout
        fig.update_layout(
            title=f"Lévy Distribution (α={alpha}, β={beta}, μ={mu})",
            xaxis_title="x",
            yaxis_title="Probability Density",
            template="plotly_white",
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99
            )
        )
        
        return fig
    
    # Activity 4: Lévy flight - Vec2d - 1 Trajectory - Callbacks
    @app.callback(
        Output("levy-flight-graph", "figure"),
        Input("levy-flight-update-button", "n_clicks"),
        [State("levy-flight-steps", "value"),
         State("levy-flight-alpha", "value"),
         State("levy-flight-mu", "value"),
         State("levy-flight-c", "value"),
         State("levy-flight-start-x", "value"),
         State("levy-flight-start-y", "value"),
         State("levy-flight-cauchy", "value")]
    )
    def update_levy_flight(n_clicks, num_steps, alpha, mu, c, start_x, start_y, cauchy_coef):
        # Generate trajectory
        trajectory = generate_levy_flight_trajectory(
            num_steps=num_steps,
            alpha=alpha,
            mu=mu,
            c=c,
            start_pos_x=start_x,
            start_pos_y=start_y,
            cauchy_coef=cauchy_coef
        )
        
        # Plot every nth point to improve performance
        plot_every_n = max(1, len(trajectory) // 500)
        
        # Create figure
        fig = go.Figure()
        
        fig.add_trace(go.Scatter3d(
            x=trajectory[::plot_every_n, 0],
            y=trajectory[::plot_every_n, 1],
            z=trajectory[::plot_every_n, 2],
            mode='lines',
            name='Lévy Flight',
            line=dict(color='blue', width=3)
        ))
        
        fig.update_layout(
            title="Lévy Flight 3D",
            scene=dict(
                xaxis_title="x_pos",
                yaxis_title="y_pos",
                zaxis_title="time",
                aspectmode='manual',
                aspectratio=dict(x=1, y=1, z=1)
            ),
            margin=dict(l=0, r=0, b=0, t=30),
            template="plotly_white"
        )
        
        return fig
    
    # Activity 5: Correlated Random Walk - Vec2d - N Trajectories - Callbacks
    @app.callback(
        Output("multi-crw-graph", "figure"),
        Input("multi-crw-update-button", "n_clicks"),
        [State("multi-crw-n-traj", "value"),
         State("multi-crw-steps", "value"),
         State("multi-crw-speed", "value"),
         State("multi-crw-start-x", "value"),
         State("multi-crw-start-y", "value")]
    )
    def update_multi_crw(n_clicks, n_traj, num_steps, speed, start_x, start_y):
        # Generate Cauchy coefficients
        cauchy_coefs = np.linspace(0.2, 0.95, n_traj)
        
        # Generate trajectories
        trajectories = generate_multiple_crw_trajectories(
            num_trajectories=n_traj,
            num_steps=num_steps,
            speed=speed,
            start_pos_x=start_x,
            start_pos_y=start_y,
            cauchy_coefs=cauchy_coefs
        )
        
        # Create color scale
        colors = [
            'blue', 'red', 'green', 'purple', 'orange', 
            'cyan', 'magenta', 'yellow', 'pink', 'brown'
        ]
        
        # Create figure
        fig = go.Figure()
        
        # Add each trajectory
        for i, (traj, coef) in enumerate(zip(trajectories, cauchy_coefs)):
            color_idx = i % len(colors)
            
            # Plot every nth point to improve performance
            plot_every_n = max(1, len(traj) // 300)
            
            fig.add_trace(go.Scatter3d(
                x=traj[::plot_every_n, 0],
                y=traj[::plot_every_n, 1],
                z=traj[::plot_every_n, 2],
                mode='lines',
                name=f'CRW, Cauchy {coef:.2f}',
                line=dict(color=colors[color_idx], width=3)
            ))
        
        fig.update_layout(
            title="N Correlated Random Walks",
            scene=dict(
                xaxis_title="x_pos",
                yaxis_title="y_pos",
                zaxis_title="time",
                aspectmode='manual',
                aspectratio=dict(x=1, y=1, z=1)
            ),
            margin=dict(l=0, r=0, b=0, t=30),
            template="plotly_white"
        )
        
        return fig
    
    # Display introduction and launch the dashboard
    print("Random Walks Dashboard is being created...")
    print("Use the 'Update' buttons on each tab to generate visualizations.")
    
    # Return the app for running
    return app

In [12]:
def run_dashboard():
    """Run the dashboard using standard Dash"""
    
    # Create the dashboard app
    app = create_dash_dashboard()
    
    # Run the app in a browser tab
    print("Dashboard will open in a new browser tab.")
    app.run_server(debug=False, port=8050)

In [13]:
if __name__ == "__main__":
    run_dashboard()

Random Walks Dashboard is being created...
Use the 'Update' buttons on each tab to generate visualizations.
Dashboard will open in a new browser tab.
